<a href="https://colab.research.google.com/github/zhiyuan-95/db-vae/blob/main/comet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet_ml --quiet
import comet_ml
from comet_ml import API
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

what I am trying to do is that I want to prove that the given method can produce a better standard cnn. which means that the validation score would reduce as the increasing epoch.

In [ ]:
#ALIASES = ['val','dbval','loss_ema', 'best_val', 'best_epoch', 'bad_val', 'stop reason','epoch']
EPOCH_PARAM_NAMES = {'batch_size','learning_rate'}

In [ ]:
def get_param_metric_records(experiments, param_names=None, last_n=20):
    """
    Build a summary DataFrame for all experiments:
      - loss, loss_ema: mean of last `last_n`
      - val, val_smooth, dbval, dbval_smooth: last value
      - stop_epoch: last logged epoch
      - best_val: maximum validation value
      - best_epoch: epoch where best_val occurred
      - drops any epoch-count parameters
    """
    rows = []

    for e in experiments:
        val, dbval = np.nan,np.nan
        loss, loss_ema, best_epoch, best_val, stop_epoch = np.nan, np.nan, np.nan, np.nan, np.nan
        stopReason = np.nan # Initialize stopReason
        for sm in e.get_metrics_summary():
          if sm['name'] == 'val':
              best_epoch = float(sm.get('stepMax', np.nan)) + 1
              best_val = float(sm.get('valueMax', np.nan))
              stop_epoch = float(sm.get('stepCurrent', np.nan)) + 1
              val = float(sm.get('valueCurrent', np.nan))
          elif sm['name'] == 'loss'       : loss = float(sm.get('valueCurrent', np.nan))
          elif sm['name'] == 'loss_ema'   : loss_ema = float(sm.get('valueCurrent', np.nan))
          elif sm['name'] == 'dbval'      : dbval = float(sm.get('valueCurrent', np.nan))
          elif sm['name'] == 'stop reason': stopReason = sm.get('valueCurrent', np.nan) # Assign stopReason here

        row = {
            "exp_key": getattr(e, "key", None),
            # aggregated metrics
            "loss"  : np.round(loss,6),    "loss_ema": np.round(loss_ema, 6),
            "val"   : np.round(val,6),     "dbval"   : np.round(dbval,6),
            # meta info
            "stop_epoch": stop_epoch,
            "best_val": best_val,
            "best_epoch": best_epoch,
        }

        # ---- collect parameters (drop epoch counts) ----
        params = {}
        for p in e.get_parameters_summary():
            name = p.get("name")
            if name in EPOCH_PARAM_NAMES:
              params[name] = p.get("valueCurrent", None)
        params.update(row)
        rows.append(params)
    df = pd.DataFrame(rows)
    df.index = df.index + 1
    return df

In [ ]:
from functools import partial

EPOCH_PARAM_NAMES = {'batch_size', 'learning_rate'}

def _experiment_to_row(e, param_names=None):
    """
    把单个 experiment 映射成一行 dict（map 的部分）
    """
    # --- 先把 metric summary 变成 name -> summary dict ---
    metric_summaries = {sm['name']: sm for sm in e.get_metrics_summary()}

    def mget(name, key, default=np.nan, cast_float=True):
        sm = metric_summaries.get(name)
        if sm is None:
            return default
        val = sm.get(key, default)
        if cast_float and val is not None and not isinstance(val, float):
            try:
                return float(val)
            except Exception:
                return default
        return val

    # ---- 从 summary 里取出你需要的指标 ----
    # val 相关
    best_epoch = mget('val', 'stepMax')
    if not np.isnan(best_epoch):
        best_epoch += 1  # comet 的 step 从 0 开始，这里转成 1-based
    stop_epoch = mget('val', 'stepCurrent')
    if not np.isnan(stop_epoch):
        stop_epoch += 1

    best_val = mget('val', 'valueMax')
    val      = mget('val', 'valueCurrent')
    loss     = mget('loss', 'valueCurrent')
    loss_ema = mget('loss_ema', 'valueCurrent')
    dbval    = mget('dbval', 'valueCurrent')

    row = {
        "exp_key": getattr(e, "key", None),
        # aggregated metrics
        "loss"      : np.round(loss, 6),
        "loss_ema"  : np.round(loss_ema, 6),
        "val"       : np.round(val, 6),
        "dbval"     : np.round(dbval, 6),
        # meta info
        "stop_epoch": stop_epoch,
        "best_val"  : best_val,
        "best_epoch": best_epoch,
    }

    # ---- 参数部分（“reduce” 汇总到 row 上）----
    # 如果传了 param_names 就用 param_names，否则用默认的 EPOCH_PARAM_NAMES
    allowed = set(param_names) if param_names is not None else EPOCH_PARAM_NAMES

    params = {}
    for p in e.get_parameters_summary():
        name = p.get("name")
        if name in allowed:
            params[name] = p.get("valueCurrent", None)

    params.update(row)
    return params


def get_param_metric_records(experiments, param_names=None, last_n=20):
    """
    Build a summary DataFrame for all experiments:
      - loss, loss_ema: 当前值（如果以后要 last_n 平均可以再扩展）
      - val, dbval: 当前值
      - stop_epoch: last logged epoch
      - best_val: maximum validation value
      - best_epoch: epoch where best_val occurred
      - stop_reason: 停止原因（如果有的话）
      - 只保留 param_names 里指定的参数（否则用默认集合）
    """
    mapper = partial(_experiment_to_row, param_names=param_names)
    # map: 每个 experiment -> 一行 dict
    rows = list(map(mapper, experiments))
    # reduce: 所有 dict 合并成一个 DataFrame
    df = pd.DataFrame(rows)
    df.index = df.index + 1  # 和你原来一样，从 1 开始
    return df


In [ ]:
def plot(comet,record,y):
    if comet.model == 'dbvae':
        params = [
            ['batch_size', 'learning_rate'],
            ['smoothing_fac','latent_dim']
        ]
        for i in range(len(params)):
            fig, ax = plt.subplots(1, 2, figsize=(10, 4))
            for j in range(len(params[i])):
                ax[j].scatter(record[params[i][j]], record[y])
                ax[j].set_xlabel(params[i][j])
                ax[j].set_ylabel(y)
                ax[j].set_title(f"{params[i][j]} vs. {y}")
                ax[j].grid(True)
            plt.tight_layout(rect=[0, 0, 1, 0.98])
            plt.show()

    elif comet.model == 'standard':
        params = ['batch_size', 'learning_rate']
        fig, ax = plt.subplots(1, 2, figsize=(10, 4))
        for i in range(len(params)):
            ax[j].scatter(record[params[i][j]], record[y])
            ax[j].set_xlabel(params[i][j])
            ax[j].set_ylabel(y)
            ax[j].set_title(f"{params[i][j]} vs. {y}")
            ax[j].grid(True)
        for sub_ax in fig.get_axes():
            sub_ax.label_outer()
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()

In [ ]:
# --- 1) pick representative trial per hyperparameter set ---
def pick_representatives(record: pd.DataFrame, by_params):
    """
    record: your model.record table that already has best_val / best_epoch per trial
    by_params: list of param columns that define 'same hyperparameter' (e.g., ['batch_size','learning_rate'])
    returns: list of row indices in `record` (int) for selected representative trials
    """
    # highest best_val wins (since higher is better)
    idx = record.groupby(by_params, dropna=False)['best_val'].idxmax()
    # idx is an Index of row labels (your record index starts from 1)
    return idx.dropna().tolist()
def metrics_at_epoch(exp, epoch, names=('loss_ema','val','dbval')):
    """
    epoch: integer epoch of interest (the 'best_epoch' from record)
    names: metric names as you logged them with experiment.log_metric(...)
    returns: dict like {'loss_ema': ..., 'val': ..., 'dbval': ..., 'loss': ...}
    """
    # build per-metric -> {step: value}
    store = {n: {} for n in names}
    for m in exp.get_metrics():
        name = m['metricName']
        if name in store:
            step = int(m['step'])
            store[name][step] = float(m['metricValue'])

    # Some notebooks log epoch starting at 0, some at 1.
    # Try exact epoch first; if missing, try epoch-1 as a fallback.
    out = {}
    for n in names:
        if epoch in store[n]:
            out[n] = store[n][epoch]
        elif (epoch - 1) in store[n]:
            out[n] = store[n][epoch - 1]
        else:
            out[n] = float('nan')
    return out
def build_summary(model, by_params):

    record = model.record
    exps = model.experiments

    # Step 1: pick the best trial for each hyperparameter group
    best_idx = pick_representatives(record,by_params)

    # Step 2: define a helper function that will run on each record
    def get_metrics_for_row(row):
        exp = exps[row.name - 1]  # because record index starts from 1
        best_epoch = int(row['best_epoch'])
        best_val = float(row['best_val'])

        # Extract metrics at that epoch
        metrics = metrics_at_epoch(exp, best_epoch-1, names=('loss_ema','val','dbval'))
        return pd.Series({
            'exp_key': row['exp_key'],
            'loss_at_best': metrics['loss_ema'],
            'val_at_best': metrics['val'],
            'dbval_at_best': metrics['dbval'],
            'best_epoch': best_epoch,
            'best_val': best_val, # it should equal val_at_best, just to check if we got right epoch properly
        })

    # Step 3: apply `map` (or technically pandas `apply`) to selected records
    selected_records = record.loc[best_idx]
    metrics_summary = selected_records.apply(get_metrics_for_row, axis=1)

    # Step 4: merge param columns + metrics summary
    param_cols = record.columns.intersection(by_params)
    summary_df = pd.concat([selected_records[param_cols].reset_index(drop=True),
                            metrics_summary.reset_index(drop=True)], axis=1)

    return summary_df.sort_values(by='val_at_best', ascending=False).reset_index(drop=True)

In [ ]:
def get_metric_names():
  metricName = []
  for x in model.experiments[0].get_metrics():
    if x['metricName'] not in metricName and x['metricName'][:3]!='sys':
      metricName.append(x['metricName'])
  print(sorted(metricName))

In [ ]:
class comet():
  def __init__(self, model, expmt=1):
    COMET_API_KEY = "ROr5Iwf4PjYLL2ZhtmHtYHhoP"
    self.api = API(COMET_API_KEY)
    self.workspace = "zhiyuan-jin"
    self.model = model
    if model == 'standard':
      self.projectName = "Lab2-Part2-standard-new-metrix"
      self.params = ['batch_size', 'learning_rate']
    elif model == 'debiasing':
      self.projectName = "Lab2-Part2-DBVAE-new-matrix"
      self.params = ['batch_size', 'learning_rate', 'smoothing_fact']
    try:
      self.experiments = self.api.get(project_name=self.projectName, workspace=self.workspace)
    except Exception as e:
      print(f"Error: Could not find project '{self.projectName}' in workspace '{self.workspace}'. Please check the project name and workspace in Comet ML.")
      raise e
    self.record = get_param_metric_records(self.experiments, self.params)
    self.top_records = None
    self.summary = build_summary(self, self.params)
  def delete_record_comet(self, keys):
    print('are you sure you want to delate following records?: ')
    for x in keys:
      print(x)
    answer = input('')
    if answer == 'yes':
      for key in keys:
        self.api.delete_experiment(key)
        print(f"Deleted experiment {key}")
      self.experiments = self.api.get(project_name=self.projectName, workspace="zhiyuan-jin")
      self.record = get_param_metric_records(self.experiments,self.params)
    else:
      print('not deleted')
  def plot_relation(self, x,y):
    if x == 'learning_rate': X = [float(l) for l in self.summary['learning_rate']]
    elif x == 'batch_size': X = [int(l) for l in self.summary['batch_size']]
    elif y == 'batch_size': Y = [int(l) for l in self.summary['batch_size']]
    else: X = self.summary[x]
    Y = self.summary[y]
    plt.figure(figsize=(10, 6))
    plt.scatter(X, Y)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f"{x} vs. {y} across Experiments")
    plt.grid(True)
    plt.show()
  def delete_nan_validation(self, confirm=True):
      nan_columns = self.v.columns[self.v.iloc[-1].isna()]
      nan_exps = self.record.loc[nan_columns.to_list()]
      if nan_exps.empty:
          print("✅ No experiments with NaN validation found.")
          return
      print(f"⚠️ Found {len(nan_exps)} experiments with NaN validation:")
      print(nan_exps[['exp_key', 'exp_name', 'validation']])
      if not confirm:
          answer = 'yes'
      else:
          answer = input("Type 'yes' to confirm deletion: ")
      if answer.lower() == 'yes':
          for key in nan_exps['exp_key']:
              try:
                  self.api.delete_experiment(key)
                  print(f"Deleted experiment {key}")
              except Exception as e:
                  print(f"❌ Failed to delete {key}: {e}")
          # Refresh records
          self.experiments = self.api.get(project_name=self.projectName, workspace=self.workspace)
          self.record = get_param_metric_records(self.experiments, self.params)
          print("✅ Refresh complete. Updated experiment list loaded.")
      else:
          print("⏩ Deletion cancelled.")
  def loss_decompose(self,exp):
    total, classification, reconstruction = [],[],[]
    for x in self.experiments[exp].get_metrics():
      if x['metricName']=='loss': total.append(float(x['metricValue']))
      elif x['metricName']=='closs_smooth': classification.append(float(x['metricValue']))
      elif x['metricName']=='rloss_smooth': reconstruction.append(0.1*float(x['metricValue']))
    plt.plot(total, label='Total Loss')
    plt.plot(classification, label='Classification Loss')
    plt.plot(reconstruction, label='Reconstruction Loss')
    plt.legend()
    plt.show()
  def val_trend(self,exp):
    loss_ema, val, dbval = [],[],[]
    for x in self.experiments[exp].get_metrics():
      if x['metricName']=='loss_ema': loss_ema.append(float(x['metricValue']))
      elif x['metricName']=='val': val.append(float(x['metricValue']))
      elif x['metricName']=='dbval': dbval.append(float(x['metricValue']))
    plt.plot(loss_ema, label='loss')
    plt.plot(val, label='val')
    plt.plot(dbval, label='dbval')
    plt.legend()
    plt.show()

In [ ]:
#model = comet('standard')
model = comet('debiasing')

In [ ]:
total, classification, reconstruction = [],[],[]
for x in model.experiments[21].get_metrics():
  if x['metricName']=='loss': total.append(float(x['metricValue']))
  elif x['metricName']=='closs_smooth': classification.append(float(x['metricValue']))
  elif x['metricName']=='rloss_smooth': reconstruction.append(0.1*float(x['metricValue']))
npclass = np.array(classification)
npvae = np.array(reconstruction)
ratio = npclass/npvae
dfratio = pd.DataFrame(ratio)
dfratio.describe()

,0
count,738.000000
mean,4.544053
std,14.708203
min,0.000183
25%,0.026504
50%,0.167231
75%,1.230046
max,151.828854


In [ ]:
model.val_trend(4)